In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

In [ ]:
pd.read_csv('Truck-Crash-Data/origPositions.csv')

We can ignore the node numbers, they're always in order

In [ ]:
origPositions = np.array(pd.read_csv('Truck-Crash-Data/origPositions.csv'))[:,1:]
displacementSim = np.zeros((126, origPositions.shape[0], 3))
for i in range(126):
    displacementSim[i] = np.array(pd.read_csv('Truck-Crash-Data/DisplacementSimulation' + str(i+1) + '.csv'))[:,1:]
displacementSim.shape

Visualize the beams before and after the simulated crash:

In [ ]:
# rescale for easier plotting 
originalMax = np.max(origPositions, axis = 0)
originalMin = np.min(origPositions, axis = 0)
displacedMax = np.max(displacementSim + origPositions, axis = (0, 1))
displacedMin = np.min(displacementSim + origPositions, axis = (0, 1))
maximum = np.maximum(originalMax, displacedMax)
minimum = np.minimum(originalMin, displacedMin)

origPositionsNorm = (origPositions - minimum)/(maximum - minimum)
displacedPositionsNorm = ((displacementSim + origPositions) - minimum)/(maximum - minimum) 

In [ ]:

def beamPlot(nodes, elevation = 30, rotation = 60, which = "both", title = ""):
    """ nodes:      array containing the x, y, z positions of the nodes
        elevation:  elevation angle in the vertical plane in degrees
        rotation:   angle in the horizontal plane in degrees
        which:      one of {"left", "right", "both"}, controls which beams are shown
        title:      string for figure title
    """
    maxLimits = [1, 1, 1]
    minLimits = [0, 0, 0]
    margin = 0.05
    fig = plt.figure(figsize=[8, 8])
    ax = fig.add_subplot(projection = "3d")
    if which == "both":
        ax.scatter(nodes[1714:,0], nodes[1714:, 1], nodes[1714:, 2], s = 5,color = "#488f31", alpha = 0.5) 
        ax.scatter(nodes[:1714,0], nodes[:1714, 1], nodes[:1714, 2], s = 5,color = "#de425b", alpha = 0.5) # we have more nodes in the red beam
    elif which == "right" :
        ax.scatter(nodes[:1714,0], nodes[:1714, 1], nodes[:1714, 2], s = 5, color = "#de425b", alpha = 0.5) 
    elif which == "left":
        ax.scatter(nodes[1714:,0], nodes[1714:, 1], nodes[1714:, 2], s = 5,color = "#488f31", alpha = 0.5)
    ax.tick_params(color = "grey")
    ax.view_init(elev = elevation,azim = rotation)
    ax.set_xlim(min(minLimits[0], 0) - margin, max(maxLimits[0], 1) + margin)
    ax.set_ylim(min(minLimits[1], 0) - margin, max(maxLimits[1], 1) + margin)
    ax.set_zlim(min(minLimits[2], 0) - margin, max(maxLimits[2], 1) + margin)
    plt.title(title, size = 20, y = 0.95, loc="left")
    plt.show()


In [ ]:
beamPlot(origPositionsNorm[:,:], 30, 60, "both", title = "Original Position")
beamPlot(displacedPositionsNorm[50,:,:], 30, 60, "both", title ="Displaced Position")

Next we want to visualize the displacement:

This is done by first normalizing appropriately and then mapping the displacement in a certain direction to a range of colors. 
We then create a sort of barcode by plotting a thin strip of color for each node. The order of the nodes is fixed so the resulting image gives an idea of the overall displacement. The goal is to have a more abstract but also more accurate way of visualizing a data point.

In [ ]:
displacementMax = np.max(displacementSim, axis = (0, 1))
displacementMin = np.min(displacementSim, axis = (0, 1))
limits = np.maximum(np.abs(displacementMax), np.abs(displacementMin)) 

displacementNorm = displacementSim/limits # sign is preserved, normalize with respect to the largest displacement in any direction

In [ ]:
def displacementPlot(data, allDirections = False, title = ""):
    """ data:           normalized displacement data, if allDirections, then of shape (N, 3), else of shape (N, )
        allDirections:  boolean, if true, the plot contains three subplots for the x, y and z displacement respectively
        title:          string for figure title
    """
    norm = mcolors.CenteredNorm(vcenter = 0, halfrange = 1) # assume normalization
    mapper = cm.ScalarMappable(norm = norm, cmap = cm.RdYlBu)
    directions = ["X", "Y", "Z"]
    if allDirections:
        fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize=(15,3))
        for i in [0, 1, 2]:
            rgbaArray = np.expand_dims(mapper.to_rgba(data[:, i]), axis = 0)
            ax[i].axis("off")
            ax[i].set_title(directions[i] + "-direction")
            ax[i].imshow(rgbaArray, extent=[0, rgbaArray.shape[1], 0, 1], aspect= "auto")
            fig.suptitle(title, size = 20, y = 1.1)
    else:
        rgbaArray = np.expand_dims(mapper.to_rgba(data), axis = 0)
        plt.axis("off")
        plt.imshow(rgbaArray, extent=[0, rgbaArray.shape[1], 0, 1], aspect= "auto")
        plt.title(title, size = 20, y = 1, loc = "left")
    plt.show()

In [ ]:
test = np.arange(start = -1, stop = 1, step = 0.01)
displacementPlot(test, title = "Range of colors from -1 to 1")

In [ ]:
%matplotlib inline
for i in np.arange(start = 0, stop = 126, step = 10):
    displacementPlot(displacementNorm[i, :, :], allDirections = True, title = "Data point " + str(i))